In [1]:
!git clone https://github.com/hungpham13/federated-learning
%cd federated-learning
!pip install -q -r requirements.txt

Cloning into 'federated-learning'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 24 (delta 9), reused 23 (delta 8), pack-reused 0
Unpacking objects: 100% (24/24), 12.16 KiB | 1.35 MiB/s, done.
/content/federated-learning
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies

In [2]:
import flwr as fl
from config import NUM_CLIENTS, DEVICE
from model import Net, VGG16
from main import simulate_cifar

## Centralize training

In [3]:
from dataloader import load_cifars
trainloaders, valloaders, testloader = load_cifars(1)

trainloader = trainloaders[0]
valloader = valloaders[0]
net = VGG16().to(DEVICE)

for epoch in range(10):
    net.train_epoch(trainloader, 1)
    loss, accuracy = net.test(valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = net.test(testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1: train loss 0.05028054490685463, accuracy 0.3868888888888889
Epoch 1: validation loss 0.04147230490446091, accuracy 0.5456
Epoch 1: train loss 0.033639177680015564, accuracy 0.6148
Epoch 2: validation loss 0.026561914348602295, accuracy 0.7034
Epoch 1: train loss 0.02576061710715294, accuracy 0.7148666666666667
Epoch 3: validation loss 0.02317628452181816, accuracy 0.744
Epoch 1: train loss 0.020628293976187706, accuracy 0.7752222222222223
Epoch 4: validation loss 0.021399199867248534, accuracy 0.771
Epoch 1: train loss 0.017199356108903885, accuracy 0.8127111111111112
Epoch 5: validation loss 0.01969984709620476, accuracy 0.7902
Epoch 1: train loss 0.0143361184746027, accuracy 0.8449333333333333
Epoch 6: validation loss 0.019189575929939747, accuracy 0.8004
Epoch 1: train loss 0.012040548026561737, accuracy 0.8689333333333333
Epoch 7: validation loss 0.017127903091907502, accuracy 0.8216
Epoch 1: train

## Fed avg with parameter initialization

In [4]:
net = VGG16().to(DEVICE)
params = net.get_parameters()

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

simulate_cifar(strategy, net)

100%|██████████| 170498071/170498071 [00:05<00:00, 29880642.23it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


INFO flwr 2023-06-09 10:44:01,919 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-06-09 10:44:04,573	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-09 10:44:06,878 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7852297422.0, 'object_store_memory': 3926148710.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7852297422.0, 'object_store_memory': 3926148710.0}
INFO flwr 2023-06-09 10:44:06,886 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-09 10:44:06,892 | server.py:269 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023-06-09 10:44:06,896 | server.py:88 | E

(launch_and_fit pid=1999) [Client 0] fit, config: {}
(launch_and_fit pid=1999) Epoch 1: train loss 0.06678900867700577, accuracy 0.1837777777777778


(pid=2089) 2023-06-09 10:44:25.237307: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

(pid=3344) 2023-06-09 10:47:09.666502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=3344) [Client 9] evaluate, config: {}


(pid=3413) 2023-06-09 10:47:16.307541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(launch_and_evaluate pid=3413) [Client 3] evaluate, config: {}


DEBUG flwr 2023-06-09 10:47:20,602 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-06-09 10:47:20,607 | server.py:147 | FL finished in 193.70295236199996
INFO:flwr:FL finished in 193.70295236199996
INFO flwr 2023-06-09 10:47:20,616 | app.py:218 | app_fit: losses_distributed [(1, 0.07478333473205566), (2, 0.07689199527104695), (3, 0.061256792783737184)]
INFO:flwr:app_fit: losses_distributed [(1, 0.07478333473205566), (2, 0.07689199527104695), (3, 0.061256792783737184)]
INFO flwr 2023-06-09 10:47:20,620 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-09 10:47:20,622 | app.py:220 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-06-09 10:47:20,624 | app.py:221 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-06-09 10:47:20,626 | app.py:222 | app_f